# `Troll Tweets Using NLP`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)

**`Help Support The Channel: Patreon @mrfugudatasci`**

**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

In [4]:
import pandas as pd


In [1]:
# Connect to Kaggle Api

from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [3]:
# all datasets with troll keyword:

api.dataset_list(search='Troll')

[vikasg/russian-troll-tweets,
 fivethirtyeight/russian-troll-tweets,
 dataturks/dataset-for-detection-of-cybertrolls,
 tannergi/microcontroller-detection,
 kapastor/democratvsrepublicantweets,
 mohammadqahtani/trolley-dilemma,
 paultimothymooney/russian-political-influence-campaigns,
 ixioixi/3-million-russian-troll-tweets-538,
 kkairu/nbc-russian-troll-tweets,
 tannergi/microcontroller-segmentation,
 wadhasnalhamdan/date-fruit-image-dataset-in-controlled-environment,
 charcoal/controllers,
 realc9der/datasets-for-controlled-over-fittinghubmap,
 kani14am/tweets1,
 hollyhetherington/russian-troll-tweets,
 omermetinn/tweets-about-the-top-companies-from-2015-to-2020,
 kishor1123/nnkdataset,
 aellatif/troll,
 jnunezp19/microcontroller-detection,
 dipinpradeep/microcontroller-detection]

In [7]:
pd.DataFrame(api.datasets_list(search='fivethirtyeight/russian-troll-tweets')).tail()

,id,ref,subtitle,tags,creatorName,creatorUrl,totalBytes,url,lastUpdated,downloadCount,...,ownerRef,kernelCount,title,topicCount,viewCount,voteCount,currentVersionNumber,files,versions,usabilityRating
0,39964,fivethirtyeight/russian-troll-tweets,3 million tweets from accounts associated with...,"[{'ref': 'russia', 'competitionCount': 0, 'dat...",Myles O'Neill,mylesoneill,183448163,https://www.kaggle.com/fivethirtyeight/russian...,2018-08-01T09:04:25.733Z,1739,...,fivethirtyeight,11,Russian Troll Tweets,2,18088,87,2,[],[],0.705882
1,40717,paultimothymooney/russian-political-influence-...,Political Propaganda on Facebook - House Intel...,"[{'ref': 'united states', 'competitionCount': ...",Paul Mooney,paultimothymooney,0,https://www.kaggle.com/paultimothymooney/russi...,2018-08-03T21:35:48.917Z,858,...,paultimothymooney,3,Internet Research Agency (IRA) Social Media Data,0,14310,32,4,[],[],0.750000
2,39892,ixioixi/3-million-russian-troll-tweets-538,This data was used in the FiveThirtyEight stor...,"[{'ref': 'arts and entertainment', 'competitio...",iXiOiXi,ixioixi,183448163,https://www.kaggle.com/ixioixi/3-million-russi...,2018-08-01T01:47:12.597Z,129,...,ixioixi,2,3 Million Russian Troll Tweets (538),0,1216,5,1,[],[],0.470588


In [10]:
# files in this dataset: total of 9 csv
api.dataset_list_files('fivethirtyeight/russian-troll-tweets').files

[IRAhandle_tweets_1.csv,
 IRAhandle_tweets_9.csv,
 IRAhandle_tweets_4.csv,
 IRAhandle_tweets_5.csv,
 IRAhandle_tweets_3.csv,
 IRAhandle_tweets_2.csv,
 IRAhandle_tweets_6.csv,
 IRAhandle_tweets_8.csv,
 IRAhandle_tweets_7.csv]

# `Create Folder to Store Troll Tweets Data`

In [11]:
import os

# define the name of the directory to be created
path = os.getcwd()+"/troll_tweets"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

Successfully created the directory /Users/zatoichi59/Desktop/troll_tweets 


# `Get all of the datasets reflecting our search:`

`api.dataset_download_files('file you want',
        'location to send file')`


In [15]:
# Download dataset: you have an 'onwer/file_name'

api.dataset_download_files('fivethirtyeight/russian-troll-tweets',
        '/Users/zatoichi59/Desktop/troll_tweets')

# you can download unzipped as well

In [16]:
# find what is inside the new folder from downloaded data:
troll_tweets_=!ls /Users/zatoichi59/Desktop/troll_tweets
troll_tweets_

['russian-troll-tweets.zip']

In [17]:
def os_dir_search(file):
    u=[]
    for p,n,f in os.walk(os.getcwd()):
        
        for a in f:
            a = str(a)
            if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
                t=p
    return t

os_dir_search(troll_tweets_[0])

'/Users/zatoichi59/Desktop/troll_tweets '

In [18]:
from zipfile import ZipFile
# specifying the zip file name 

file_name=os_dir_search(troll_tweets_[0])+ '/' + troll_tweets_[0]
  
# opening the zip file in READ mode 
with ZipFile(file_name, 'r') as zip: 
    # printing all the contents of the zip file 
    zip.printdir() 
  
    # extracting all the files 
    print('Extracting all the files now...') 
    zip.extractall('/Users/zatoichi59/Desktop/troll_tweets') 
    print('Done!')



File Name                                             Modified             Size
IRAhandle_tweets_1.csv                         2019-10-03 00:12:48     87874842
IRAhandle_tweets_2.csv                         2019-10-03 00:12:56     89384804
IRAhandle_tweets_3.csv                         2019-10-03 00:13:02     87696348
IRAhandle_tweets_4.csv                         2019-10-03 00:13:10     87771344
IRAhandle_tweets_5.csv                         2019-10-03 00:13:18     79412644
IRAhandle_tweets_6.csv                         2019-10-03 00:13:24     77962778
IRAhandle_tweets_7.csv                         2019-10-03 00:13:32     82408170
IRAhandle_tweets_8.csv                         2019-10-03 00:13:38     85947534
IRAhandle_tweets_9.csv                         2019-10-03 00:13:46      9444208
Extracting all the files now...
Done!


In [22]:
print('Location of my Insurance Folder: ',os_dir_search(troll_tweets_[0]))
print('-------------------------------')
print('Files in Insurance Folder: \n',os.listdir(os_dir_search(troll_tweets_[0])))

Location of my Insurance Folder:  /Users/zatoichi59/Desktop/troll_tweets
-------------------------------
Files in Insurance Folder: 
 ['IRAhandle_tweets_1.csv', 'IRAhandle_tweets_2.csv', 'IRAhandle_tweets_3.csv', 'IRAhandle_tweets_4.csv', 'IRAhandle_tweets_5.csv', 'IRAhandle_tweets_6.csv', 'IRAhandle_tweets_7.csv', 'IRAhandle_tweets_8.csv', 'IRAhandle_tweets_9.csv', 'russian-troll-tweets.zip']


In [24]:
# combine the files into a list

my_dir=os_dir_search(troll_tweets_[0])
trolls_=[]
for file in os.listdir(my_dir):
    trolls_.append(file)

# pd.read_csv(trolls_[2]).head()

In [36]:
# Create DF from each file:

k=[]
for i in trolls_[:-1]: # last file is .zip exclude it
    # read each csv into DF
    kaggle_tweets=pd.read_csv(os_dir_search(troll_tweets_[0])+'/'+str(i))
    # save all DF's into list
    k.append(kaggle_tweets)


In [38]:
# Almost 3 Million Troll Tweets, combined from 9 files:

mega_troll_df=pd.concat(k,axis=0,ignore_index=True)

mega_troll_df.tail()

,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category
2973366,3.093874e+09,_YOUR_LIFESTYLE,Санкционный пинг-понг http://t.co/Jz06cndR8q,Unknown,Uzbek,9/7/2015 1:31,9/7/2015 1:31,60,4,412,NaN,Russian,0,0,NonEnglish
2973367,3.093874e+09,_YOUR_LIFESTYLE,Горизонтальный кальян NanoSmoke http://t.co/rL...,Unknown,Russian,9/8/2015 14:49,9/8/2015 14:49,60,4,416,NaN,Russian,0,0,NonEnglish
2973368,3.093874e+09,_YOUR_LIFESTYLE,Фотограф Kai-Hendrik Schroeder http://t.co/q0x...,Unknown,Russian,9/9/2015 18:52,9/9/2015 18:52,60,4,418,NaN,Russian,0,0,NonEnglish
2973369,3.093874e+09,_YOUR_LIFESTYLE,К чему стремится компания Netflix (США) http:/...,Unknown,Russian,9/9/2015 22:26,9/9/2015 22:27,60,4,419,NaN,Russian,0,0,NonEnglish
2973370,3.093874e+09,_YOUR_LIFESTYLE,В Петербурге появилось вегетарианское кафе в к...,Unknown,Russian,9/9/2015 8:17,9/9/2015 8:17,60,4,417,NaN,Russian,0,0,NonEnglish


In [39]:
mega_troll_df.columns

Index(['external_author_id', 'author', 'content', 'region', 'language',
       'publish_date', 'harvested_date', 'following', 'followers', 'updates',
       'post_type', 'account_type', 'new_june_2018', 'retweet',
       'account_category'],
      dtype='object')

# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

**`Help Support The Channel: Patreon @mrfugudatasci`**

**`Or Consider: Buy Me A Coffee @mrfugudatasci`**

# `Citations & Help:`

# ◔̯◔

`Kaggle`

https://medium.com/@jeff.daniel77/accessing-the-kaggle-com-api-with-jupyter-notebook-on-windows-d6f330bc6953

https://github.com/Kaggle/kaggle-api

https://technowhisp.com/kaggle-api-python-documentation/

https://www.kaggle.com/docs/api#interacting-with-datasets

https://www.geeksforgeeks.org/working-zip-files-python/

https://medium.com/analytics-vidhya/fetch-data-from-kaggle-with-python-9154a4c610e3

https://stackabuse.com/creating-and-deleting-directories-with-python/

https://www.kaggle.com/paultimothymooney/exploring-the-kaggle-api

https://stackoverflow.com/questions/50836642/i-get-a-bash-command-not-found-when-trying-to-use-the-kaggle-api

https://github.com/Kaggle/kaggle-api/issues/29

`NLP`

